# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from IPython import display
import warnings

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download(['punkt', 'wordnet', 'stopwords'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("Select * from InsertTableName", engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Tokenize the text column in the dataset using functions like
    word_tokenize, WordNetLemmatizer and remove the words contained
    in the list of English stopwords
    
    Input
        Text - The text column containing the message information in the dataframe
        
    Output
        List of strings parsed from the text column
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Text Normalization
    
    tokens = word_tokenize(text) # Text Tokenization
    
    lemmatizer = WordNetLemmatizer() # Lemmatize words and remove stop words
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result
    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 40)

In [11]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
def evaluation_report(actual, predicted, col_names):
    """
    Calculate the evaluation metrics for the model. F1 Score, Precision & Recall
    
    Input
        actual - Array containing the actual labels
        predicted - Array containing the predicted labels
        col_names - List of strings containing names for each of the predicted fields
    
    Output
        metrics_dataframe - Dataframe containing the F1 score, Precision & Recall
                            for the provided actual and predicted fields
    """
    metrics = []
    for i in range(len(col_names)):
        f1 = f1_score(actual[:, i], predicted[:, i], average='micro')
        precision = precision_score(actual[:, i], predicted[:, i], average='micro')
        recall = recall_score(actual[:, i], predicted[:, i], average='micro')
        
        metrics.append([f1, precision, recall])
    
    metrics = np.array(metrics)
    metrics_dataframe = pd.DataFrame(data = metrics, index = col_names, columns = ['F1', 'Precision', 'Recall'])
    
    return metrics_dataframe

In [26]:
col_names = list(y.columns.values)
y_train_pred = pipeline.predict(X_train)

print(evaluation_report(np.array(y_train), y_train_pred, col_names))

                              F1  Precision    Recall
related                 0.990947   0.990947  0.990947
request                 0.986777   0.986777  0.986777
offer                   0.998372   0.998372  0.998372
aid_related             0.983165   0.983165  0.983165
medical_help            0.987997   0.987997  0.987997
medical_products        0.991201   0.991201  0.991201
search_and_rescue       0.994507   0.994507  0.994507
security                0.994609   0.994609  0.994609
military                0.994507   0.994507  0.994507
child_alone             1.000000   1.000000  1.000000
water                   0.994812   0.994812  0.994812
food                    0.995067   0.995067  0.995067
shelter                 0.990693   0.990693  0.990693
clothing                0.997559   0.997559  0.997559
money                   0.995779   0.995779  0.995779
missing_people          0.997050   0.997050  0.997050
refugees                0.993032   0.993032  0.993032
death                   0.99

In [28]:
y_pred = pipeline.predict(X_test)

print(evaluation_report(np.array(y_test), y_pred, col_names))

                              F1  Precision    Recall
related                 0.807751   0.807751  0.807751
request                 0.880989   0.880989  0.880989
offer                   0.995728   0.995728  0.995728
aid_related             0.756790   0.756790  0.756790
medical_help            0.921575   0.921575  0.921575
medical_products        0.951633   0.951633  0.951633
search_and_rescue       0.972994   0.972994  0.972994
security                0.982911   0.982911  0.982911
military                0.971926   0.971926  0.971926
child_alone             1.000000   1.000000  1.000000
water                   0.950565   0.950565  0.950565
food                    0.933781   0.933781  0.933781
shelter                 0.928746   0.928746  0.928746
clothing                0.987946   0.987946  0.987946
money                   0.978334   0.978334  0.978334
missing_people          0.989777   0.989777  0.989777
refugees                0.966585   0.966585  0.966585
death                   0.95

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.